In [1]:
# Import needed packages
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
import csv
import sys
import pandas_datareader as pdr
import datetime


In [2]:
# Obtain stock tickers

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')
#print(data)
table = data[0]
#table.head()
tickers = table['Symbol'].tolist()
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'AIG', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ANET', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', '

In [9]:
# Get stock prices and returns
startdate = datetime.datetime(2020,3,23)
enddate = datetime.datetime(2020,3,27)

temp_tickers = ['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 
                'AFL', 'A', 'AIG', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 
                'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AMT', 'AWK', 'AMP', 'ABC', 'AME',
                'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ANET', 'AIZ', 'T',
                'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY',
                'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 
                'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 
                'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB',
                'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 
                'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 
                'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 
                'DLR', 'DFS', 'DISCA', 'DISCK', 'DISH', 'DIS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 
                'DRE', 'DD', 'DXC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ENPH', 'ETR', 'EOG', 
                'EPAM', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'RE', 'EVRG', 'ES', 'EXC', 'EXPE', 'EXPD', 'EXR',
                'XOM', 'FFIV','FDS', 'FAST']

file = open("stocks.csv",'w',newline='')
for ticker in tickers:
    try:
        st = pdr.get_data_yahoo(ticker,start=startdate,end=enddate)
        st['pct_change'] = st['Adj Close'].pct_change(4)
        #print(ticker + str(st['Adj Close']) + str(st['pct_change']))
        stockdata = csv.writer(file)
        stockdata.writerow([ticker,st['pct_change'].iloc[-1]])
    except:
        pass

file.close()


In [10]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
consumer_key = 'lLoOz2cbplDwX3vTzZFv2Hyqz'
consumer_secret = 'Yp5YFPvY3ISI2KUVQC6HAqYjDCUWw5na3IFrCvTVoTuvF9QddN'
access_key= '3028339548-i1TWGaqfItqlybCtkJZUQ7km95pylMZl8zPQBLb'
access_secret = 'llC8DaUitlD5wMl0FMqSxR3L78q0OaF56oazKJF5Hqnnr'

#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
api = tweepy.API(auth)

file = open("searchTweets.csv","w",newline='')
alltweets = csv.writer(file)

for ticker in tickers:
    query = "$"+ticker + " -filter:retweets -filter:replies"
    tweets = tweepy.Cursor(api.search_tweets,q=query,lang='en',tweet_mode='extended').items(10)
    for tweet in tweets:
        status=tweet._json
        created_at = status['created_at']
        tweet_id = status['id_str']
        tweet_text = status['full_text']
        tweet_text = re.sub(r'\W', ' ',tweet_text)
        print(tweet_text)
        try: 
            alltweets.writerow([ticker,created_at,tweet_id, tweet_text])
        except UnicodeEncodeError:
            pass

file.close()


 MMM 13 has crossed below    48 and ERROR retrieving 200 data  check manually  for the 60m intverval  03 45 02 PM
 MMM  Waiting for BUY signal on    MMM    Register for 7 Day Trial Access at https   t co WBARPaK0Cc   https   t co C6OxCBuvh9
 ENB  MMM  MO  NVS  UGI   5  Yielding Dividend Aristocrats You ll Want To Own In The Coming Bear Market https   t co TL0TMYQ1g0 https   t co I192pEf9v7
 ENB  MMM  MO NEW ARTICLE   5  Yielding Dividend Aristocrats You ll Want To Own In The Coming Bear Market https   t co W8c9XMhzus Get all the latest  ENB related news here   https   t co Yuyi4PQvkZ
 FOR  A  RATE  I  CRY  TWO  AT  A  TIME  MMM 
3M Company  NYSE   MMM    Peering Into 3M Company s Recent Short Interest https   t co Fjt845zO4l
 MMM  NP  SWM NEW ARTICLE   A Georgia Wedding Between 2 Specialty Material Companies https   t co Wh6Dca5kyp Get all the latest  MMM related news here   https   t co vm9QziaCd9
A Georgia Wedding Between 2 Specialty Material Companies  NP  SWM  MMM  stocks  investin

 Cybersecurity Solution  Stocks in the news    OTC PINK   GTCH   NYSE   S   NasdaqGS   MNDT   NasdaqGS   GOOG   NYSE   ACN   SentinelOne  Mandiant  Google  Accenture  Stocks  markets  Finance  Business  Investing  tech  technology https   t co bQZCDGdPvU https   t co vXHZPMVI6X
 ACN If you really want to make huge profits on trading then check out this chat https   t co 0DCg8pjRkf
 GTCH  Cybersecurity Solutions  Stocks in the news   OTC PINK   GTCH   NYSE   S   NasdaqGS     GTCH  S  MNDT  GOOG  ACN https   t co YnJ2HqEoiM
 GTCH  Cybersecurity Solutions  Stocks in the news   OTC PINK   GTCH   NYSE   S   NasdaqGS     GTCH  S  MNDT  GOOG  ACN https   t co v8GksQQf2z
In Case you Missed it   Cybersecurity Solutions  Stocks in the news   OTC PINK   GTCH   NYSE   S   NasdaqGS   MNDT   NasdaqGS   GOOG   NYSE   ACN   SentinelOne  Mandiant  Google  Accenture  Stocks  markets  Finance  Business  Investing  tech  technology https   t co bQZCDGdPvU https   t co 599ubFmByD
Cybersecurity Solutions St

 AAP If you really want to make huge profits on trading then check out this chat https   t co 0DCg8pjRkf
INIO s Gold Daily Stock Screener   QRVO   CRAI   THRM   HSIC   CHUY   MKSI   ACLS   HI   RUTH   EVTC   MOH   AAP   NXPI   G
 AAP Best Room on the net   Stay Turned https   t co n5TZnOE5Cp
Tekmetric Is Now Advance Auto Parts  Recommended Shop Management System https   t co LOPcSpJ6eU   tekmetric  AdvanceAuto  AAP  aftermarket  autoparts  autocare https   t co IVcS9JmXjg
 AAP All that you need to know  Shorts haven t covered a single share  https   t co n5TZnODxMR
 AAP  BBY  BX  DLR  EPD  MMM  MS  QSR  TROW  UPS   My Top 10 High Yield Dividend Stocks For April 2022 https   t co 5GF3dk98nP https   t co 9yxkoS2ceG
 AAP  BBY  BX  DLR  EPD  MMM  MS  QSR  TROW  UPS   My Top 10 High Yield Dividend Stocks For April 2022 https   t co e8URFykIIl https   t co QAY5Sx4FFv
 AAP  BBY  BX NEW ARTICLE   My Top 10 High Yield Dividend Stocks For April 2022 https   t co tL5QM5WDo0 Get all the latest  AA

Akamai Technologies  Inc  traded 7  above it s 50 DMA and 7  above it s 200 DMA  AKAM
New 52 Week High today   AZN  BREZ  IGF  STE  BMY  RACYU  VOLT  FXU  MUSA  VCXB  REGN  CEG  EXLS  AKAM  PST  THCA  RAAX  FTXG  TYO  TBX     https   t co 4Iw0QeR3p8
 AKAM Implied Volatility expiration 04 08 2022  maxpain  options https   t co 0TAPO3Z1oj https   t co auor370B8x
 NASDAQ  NewHigh Count Technology  TENB 2  AKAM 2  QLYS 2  CNXN 2  LLNW 2  EXLS 1  GVP 1  JKHY 1  AOSL 1  COHR 1  From 2022 03 29 
Akamai Technologies  Inc current quarter revenue growth YoY of 7 00  ranked 2639 out of 5728 companies in our database      https   t co Cr9bZVzF6Z  AKAM        https   t co ieTruwLxWJ
Akamai Technologies  Inc current quarter revenue growth YoY of 7 00  ranked 2639 out of 5728 companies in our database      https   t co Cr9bZVzF6Z  AKAM        https   t co tDmgbJqofE
Call Put Highest   Volume Changes   Calls   COOP   TWTR   MFA   KTOS   ATER Puts   TWTR   RAD   AKAM   FSR   JMIA
 AKAM  110 00 Aug 19 2

Allegion plc current quarter revenue growth YoY of  2 50  ranked 3322 out of 5728 companies in our database      https   t co xTg89t95fD  ALLE        https   t co AS4Q5U9Z5M
Rose Above Previous Day s High today   XDEC  HTPA  DTSS  QED  ATY  FSMD  CVY  SYNL  CM  GSID  BLKC  ALLE  CNCR  ARTE  CLSM  IBDN  TCX  BWAY  IMV  GSEE     https   t co TBYRpvrFqi
 ALLE Allegion 2022 Health Care Trends Report Uncovers How Facilities Are Using Technology to Meet the Demands of Patients and Staff https   t co Wkenzh7ovw
 ALLE   Allegion 2022 Health Care Trends Report Uncovers How Facilities Are Using Technology to Meet the Demands of Patients and Staff  https   t co Yn1ajAW4W8  45 seconds delayed 
 ALLE Allegion 2022 Health Care Trends Report Uncovers How Facilities Are Using Technology to Meet the Demands of Patients and Staff   https   t co Ny0tB8qI4E
 ALLE Allegion 2022 Health Care Trends Report Uncovers How Facilities Are Using Technology to Meet the Demands of Patients and Staff https   t co cfai

...

 DD Wealth is your right   tokensale  digitalcurrency  coin  token  tokens  digitalcurrency  nftnews https   t co dhAoTZLXCD
Is there a test to determine if blood clots were caused by vaxx or BPS     XLV  CRM  VHT  BLK  STT  JPM  BK  BAC  NTRS  NBIM  MS  WFC  GS  JNJ  MRNA  PFE  BNTX  AZN  GLW  BDX  RVP  MRK  LLY  REGN  GSK  AMGN  NVAX  DD  BTI  PM  SAVE  ULCC  MDLZ  CPB https   t co 62sJpfkBgb
Should those with  female  reproductive systems  be made aware to avoid BPA to improve fertility     XLV  CRM  VHT  BLK  STT  JPM  BK  BAC  NTRS  NBIM  MS  WFC  GS  JNJ  MRNA  PFE  BNTX  AZN  GLW  BDX  RVP  MRK  LLY  REGN  GSK  AMGN  NVAX  DD  BTI  PM  SAVE  ULCC  MDLZ  CPB https   t co 2G5ZBslN9s
Maybe avoid BPA to improve fertility  Or is the damage permanent     XLV  CRM  VHT  BLK  STT  JPM  BK  BAC  NTRS  NBIM  MS  WFC  GS  JNJ  MRNA  PFE  BNTX  AZN  GLW  BDX  RVP  MRK  LLY  REGN  GSK  AMGN  NVAX  DD  BTI  PM  SAVE  ULCC  MDLZ  CPB https   t co 4oujkmtqs5
Should we increase awareness of gend

   ECL price target update    Initiated by Atlantic Securities Price target   210  Day quote change   180 24  0 11   Target upside  16 51   Published  April 5  2022 https   t co xfTYbX3VDj
Daily Chemical Reaction  Hot N Cold   US PE Price Premiums Have Turned To Discounts  High Prices Abroad Benefit US Producers   https   t co OlWYy1Q332  chemicals  energy  sustainability  DOW  LYB  XOM  RDS  WLK  CVX  SRE  TTE  TALO  ATC  ALB  AQUA  KOP  ECL  APD  GEVO https   t co W2kk8qpmVZ
Today Top Flow in basic  materials sector  Buy Flow  amp  Sell Flow  X  DOW  BLL  AVY  FCX  ATI  PX  DD  VMC  PPG  MOS  NUE  CF  SHW  LYB  NEM  AA  ECL  SEE  EMN  stock  stocks  StockMarket  Investment  investing https   t co Gr3YvQQ4OY https   t co K8t0m0OoIo
   ECL price target update    Initiated by Atlantic Securities Price target   210  Day quote change   181 23  0 63   Target upside  15 87   Published  April 5  2022 https   t co xfTYbX3VDj
 ECL   New Insider Filing on ECOLAB INC  s  Director  SUZANNE M VAUT

Check Out What Whales Are Doing With   ENPH   https   t co FKoZwzTyY1
They are buying the April and May Calls  Keep an eye on it   TAN  ICLN  ENPH  SEDG https   t co ZIjibaBRi6
 ENPH  Waiting for SHORT signal on     ENPH   Register for 7 Day Trial Access at https   t co WBARPaK0Cc   https   t co ieFYMNlqri
Your broadening formations on the 60s   What s the next 2   theStrat   PLUG  FB  ENPH https   t co 7dMvAQjqdE
 ENPH Best Room on the net   Stay Turned https   t co n5TZnOE5Cp
Trending Stocks on  XTRADER  1   TSLA   2   SEDG   3   MSFT   4   TWTR 5   AAPL 6   DWAC 7   BABA 8   PLUG 9   ENPH 10   PWR   Sorted by most recent       https   t co JsCIco4BtE https   t co DpwUr4YhtR
Trade Recap on all my trades today  Red ovals Short and Green ovals Long     MU  TWTR  LLY  ENPH  Stopped out on  MRNA for break even because it was just chopping at support    optionstrading  daytrading https   t co eymZaTM1nz
Enphase Energy  Inc current quarter revenue growth YoY of 55 80  ranked 819 out of 572

Winfield Associates Inc  has filed Form 13F for Q1 2022 Opened NEW positions in  AZN  BATT  CIEN  COP  EQR  F  KBE  LAC  RELY  SCHX  TRV  Try Insider Forms mobile app  get realtime notifications and more  https   t co t5CkFUZh7T
 EQR  NYSE  Stock Rating Changed to Strong Buy  Equity Residential  gt  gt  https   t co LzRdUTXfGX
Scan results   Fell Below 50 DMA today   M2R  GTK  DDH  LMG  TTB  QGL  FRB  POS  MDI  4DS  FME  CTP  EQR  VMC  ID8  GCY     https   t co UbmYzm1ZuJ
Rose Above Previous Day s High today   DVA  A  ANTM  TFX  IQV  ZTS  MRK  BIO  CRL  TMO  CTLT  PEAK  EQR  GPN  ACN  TDG  REXR  RYN  FCPT  LSI     https   t co TBYRpvrFqi
Equity Residential    EQR    QUOTE  EQR was DOWN   on Fri Apr 01 Closed at 92 16 USD for a 0 00  loss from previous close          ANALYSIS Equity Residential should be nationalized      
Rose Above Previous Day s High today   CWX  E25  CNB  GNP  EQR  GL1  29M  QML  ARR  DCX  GT1  MNRS  BBUS  LEL  WMG  IXU     https   t co Sr2t6babZ6
 EQR Best room on 

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded

In [ ]:
# Sentiment analysis

file = open("searchTweets.csv","r")
alltweets = csv.reader(file)

sentiment_file = open("sentimenttweets.csv","w",newline='')
sntTweets=csv.writer(sentiment_file)

for row in alltweets:
    blob = TextBlob(row[3])
    if blob.sentiment.polarity>0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"positive"])
    elif blob.sentiment.polarity<0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"negative"])     
    elif blob.sentiment.polarity==0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"neutral"])    
        
file.close()
sentiment_file.close()